In [14]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import seaborn as sns; sns.set(style='white')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from sklearn import decomposition
from sklearn import datasets
from mpl_toolkits.mplot3d import Axes3D
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')
import random
from sklearn.metrics import precision_score, accuracy_score
from pandas.api.types import is_numeric_dtype

data = pd.read_csv('/home/administrator123/Akanksha/Company_Work/datasetcpulogs/Data/CpuLogData.csv')
# data = pd.read_csv('https://raw.githubusercontent.com/prayas2409/logs/master/CpuLogData.csv')
data['Dates'] = pd.to_datetime(data['DateTime']).dt.date
data['Time'] = pd.to_datetime(data['DateTime']).dt.time


u_name = ['mohitkr1301@gmail.com', 'sapnapatil344@gmail.com', 'akankshakaple@gmail.com'
         ,'you@example.com', 'honeykrsingh16@gmail.com', 'kiranraikar777@gmail.com'
          , 'kadamsagar039@gmail.com', 'singh.saurabh3333@gmail.com']
for i in u_name:
    data.drop(data[data['user_name'] == i].index, inplace=True)

    
feature_eng_col = ['Cpu Working Time', 'Cpu idle Time', 'number of software interrupts since boot'
                   ,'number of interrupts since boot', 'disk_read_count', 'disk_write_count'
                   ,'disk_read_bytes', 'disk_write_bytes', 'time spent reading from disk'
                   ,'time spent writing to disk', 'time spent doing actual I/Os'
                   ,'number of bytes sent', 'number of bytes received'
                   ,'number of packets sent', 'number of packets recived']

# To be checked again

# Remove constant data
constant_col = ['Cpu Count', 'Usage Cpu Count ', 'number of system calls since boot'
                , 'system_total_memory', 'total number of errors while receiving'
                ,'total number of errors while sending','total number of incoming packets which were dropped'
                ,'total number of outgoing packets which were dropped', 'disk_total_memory']

# Remove Inter dependent data
inter_dependet_cols = ['system_free_memory', 'disk_free_memory', 'system_avalible_memory', 'system_used_memory']

# Remove object data 
object_col = ['DateTime', 'Dates', 'Time', 'boot_time']

def remove_col(data,col_list):
    data.drop(columns=col_list, inplace=True)
    return data
data = remove_col(data, inter_dependet_cols)
# data.columns
data = remove_col(data, constant_col)

def feature_engineering(feature_eng_col, data):    
    user_name = data['user_name'].unique()
    final_df = pd.DataFrame()
    # print(user_name)
    for u_name in user_name:
        df = data[data['user_name'] == u_name]
        df.sort_values('DateTime', inplace=True)
        df.reset_index(drop=True, inplace=True)
        for col in feature_eng_col:
            l1 =[]
            l1.append(df[col].iloc[0])
            for index in range(1,len(df)):

                if (df["Dates"].iloc[index] == df["Dates"].iloc[index-1]):
                    # As when curr smaller than prev
                    if (df[col].iloc[index]-df[col].iloc[index-1]) <= 0:
                        l1.append(df[col].iloc[index])
                        # curr > prev
                    elif (df[col].iloc[index]-df[col].iloc[index-1]) > 0:
                        l1.append(df[col].iloc[index]-df[col].iloc[index-1])

                else:
                    l1.append(df[col].iloc[index])
            df[col+"_fe"]=l1
        final_df = final_df.append(df)
        for col in feature_eng_col:
            final_df.drop(columns=col, inplace=True)
    return final_df

final_df = feature_engineering(feature_eng_col, data)

data.shape, final_df.shape

In [52]:
from sklearn.preprocessing import StandardScaler 
def std(data):
    # standardize the data attributes
    for col in data.columns:
        if data[col].dtype == 'object':
            data = data.drop(columns=col)
    sc = StandardScaler()
    standardized_X = sc.fit_transform(data)
    new_data = pd.DataFrame(standardized_X, columns=data.columns)
    
    return new_data
new_data = std(final_df)

In [54]:
new_data['Dates'] = list(final_df['Dates'])
new_data['user_name'] = list(final_df['user_name'])
new_data['Time'] = list(final_df['Time'])

In [69]:
# new_data.user_name.value_counts()

In [68]:
# target = (np.sqrt((df['Humidity'])))
# print ('Skewness is', target.skew())
# sns.distplot(target)
# new_data.dtypes

In [60]:
def visualization(data):
    dt = data[data['user_name']=='sheetalbedarkar96@gmail.com']
    dt2 = dt[dt['Dates']==dt['Dates'].unique()[2]]
    
    for i in range(1,len(dt2.columns)):
        if final_df[dt2.columns[i]].dtype != 'object':
            print(dt2.columns[i])
#             plt.figure(figsize=(15,7))
            sb.lineplot(x='Time', y=dt2.columns[i], data=dt2)
            plt.xticks(rotation=90)
            plt.show()
        
visualization(new_data)

In [28]:
# for col in final_df.columns:
#     if final_df[col].dtypes == 'object':
#         final_df.drop(columns=col, inplace=True)

In [29]:
# final_data.shape

In [30]:
# final_df.duplicated().sum()

In [1]:
# Remove the outlier from the whole dataset
# def remove_outlier(df):
#     low = .20
#     high = .80
#     quant_df = df.quantile([low, high])
#     for name in list(df.columns):
#         if is_numeric_dtype(df[name]):
#             df = df[(df[name] > quant_df.loc[low, name]) & (df[name] < quant_df.loc[high, name])]
#     return df

# df= remove_outlier(final_df)
# # sb.boxplot(df)
# plt.figure(figsize=(15,7))
# # plt.xticks(rotation='vertical')
# s = df.boxplot()
# s.set_xticklabels(df.columns,rotation=90)

In [4]:
# df.isna().sum()

In [2]:
# from sklearn.preprocessing import StandardScaler 
# def std(data):
#     # standardize the data attributes
#     for col in data.columns:
#         if data[col].dtype == 'object':
#             data = data.drop(columns=col)
#     sc = StandardScaler()
#     standardized_X = sc.fit_transform(data)
#     new_data = pd.DataFrame(standardized_X, columns=data.columns)
    
#     return new_data
# new_data = std(df)

In [3]:
# from sklearn.cluster import KMeans
# wcss = []

# X = np.array(new_data)

# for i in range(1, 11):
#     kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
#     kmeans.fit(X.reshape(-1,1))
#     wcss.append(kmeans.inertia_)
# plt.plot(range(1, 11), wcss)
# plt.title('The Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()

# kmeans = KMeans(n_clusters = 4, init = 'k-means++', random_state = 42)
# y_kmeans = kmeans.fit_predict(new_data)
# new_data['y'] = y_kmeans

# sb.countplot(x=y_kmeans, data=new_data)


In [ ]:
# # sb.boxplot(final_df)
# from pandas.api.types import is_numeric_dtype
# def remove_outlier(df, name):
#     low = .25
#     high = .75
#     quant_df = df.quantile([low, high])
# #     for name in list(df.columns):
# #         if is_numeric_dtype(df[name]):
#     for i in range(len(df)):
#         if df[name].iloc[i]  > quant_df.loc[low, name] or df[name].iloc[i] < quant_df.loc[high, name]:
# #             print(df[name].iloc[i])
#             df[name].iloc[i] = np.average(df[name])
# #             print(df[name].iloc[i])
#     return df
# l = ['system_active_memory','disk_used_memory','system_inactive_memory','system_buffers_memory',
#        'system_cached_memory','system_shared_memory']
# # for i in l:
# #     print(i)
# df= remove_outlier(new_data,'system_active_memory')
# #     sb.boxplot(df)
# plt.figure(figsize=(15,7))
# plt.xticks(rotation='vertical')
# s = df.boxplot()
# s.set_xticklabels(df.columns,rotation=90)

In [ ]:
# low = .20
# high = .80
# name = 'system_active_memory'
# quant_df = df.quantile([low, high])
# print(quant_df.loc[low, name], quant_df.loc[high, name])
# np.average(new_data[name])

In [ ]:
# # Remove the outlier from the whole dataset
# def remove_outlier(df):
#     low = .25
#     high = .75
#     quant_df = df.quantile([low, high])
#     for name in list(df.columns):
#         if is_numeric_dtype(df[name]):
#             df[name] = df[(df[name] > quant_df.loc[low, name]) & (df[name] < quant_df.loc[high, name])]
#     return df

# df= remove_outlier(new_data)
# plt.figure(figsize=(15,7))
# plt.xticks(rotation='vertical')
# s = df.boxplot()
# s.set_xticklabels(df.columns,rotation=90)

In [ ]:
# df['system_active_memory'], final_df['system_active_memory']
# for col in new_data.columns:
#     if new_data[col].isna():
#         new_data.drop(columns=col, inplace=True)

In [ ]:
# np.average(new_data['Cpu Working Time_fe'])

In [ ]:
# from sklearn.cluster import KMeans
# wcss = []

# X = np.array(new_data)

# for i in range(1, 11):
#     kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
#     kmeans.fit(X.reshape(-1,1))
#     wcss.append(kmeans.inertia_)
# plt.plot(range(1, 11), wcss)
# plt.title('The Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()

# kmeans = KMeans(n_clusters = 4, init = 'k-means++', random_state = 42)
# y_kmeans = kmeans.fit_predict(new_data)
# new_data['y'] = y_kmeans

# sb.countplot(x=y_kmeans, data=new_data)


In [1]:
# plt.figure(figsize=(10,10))
# cor = new_data.corr() #Calculate the correlation of the above variables
# sb.heatmap(cor, square = True) #Plot the correlation as heat map

In [ ]:
# df_cluster = final_df
# df_cluster.shape, final_df.shape
# d = final_df

In [ ]:
# kmeans = KMeans(n_clusters = 3, init = 'k-means++', random_state = 42)
# y_kmeans = kmeans.fit_predict(final_df)
# final_df['y'] = y_kmeans

# sb.countplot(x=y_kmeans, data=final_df)

In [ ]:
# plt.figure(figsize=(10,10))
# sb.heatmap(final_df.corr())

In [ ]:
# plt.figure(figsize=(15,8))
# sb.boxplot(data=final_df)
# plt.xticks(rotation=90)

In [ ]:
#import important libraries.
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from pandas.api.types import is_numeric_dtype
# import seaborn as sns
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, accuracy_score
# def remove_outlier(df):
#     low = .25
#     high = .75
#     quant_df = df.quantile([low, high])
#     for name in list(df.columns):
#         if is_numeric_dtype(df[name]):
#             df = df[(df[name] > quant_df.loc[low, name]) & (df[name] < quant_df.loc[high, name])]
#     return df

# df= remove_outlier(final_df)
# sns.boxplot(df)

In [ ]:
# for col in final_df.columns:
# #     plt.hist(x=col)
#     sb.distplot(final_df[col])
#     plt.xticks(rotation=90)
#     plt.show()

In [ ]:
# from sklearn.preprocessing import StandardScaler

# # No Feature Scaling as library does it
# from sklearn.preprocessing import StandardScaler
# sc_X = StandardScaler()
# X = sc_X.fit_transform(final_df)



In [ ]:
# from sklearn.cluster import KMeans
# wcss = []
# X = np.array(final_df)

# for i in range(1, 11):
#     kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
#     kmeans.fit(X)
#     wcss.append(kmeans.inertia_)
# plt.plot(range(1, 11), wcss)
# plt.title('The Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()

# kmeans = KMeans(n_clusters = 3, init = 'k-means++', random_state = 42)
# y_kmeans = kmeans.fit_predict(final_df)
# final_df['y'] = y_kmeans

# sb.countplot(x=y_kmeans, data=final_df)
